In [1]:
import pandas as pd
from gurobipy import GRB, Model, quicksum

import utils.util as utils

In [2]:
cities_df = pd.read_csv('data/cities.csv')
cities_df

,Area,City,x,y,TB/s
0,1,Warszawa,100,65,-
1,1,Skierniewice,92,62,197
2,1,Lódz,85,58,317
3,1,Konin,68,64,236
4,1,Sieradz,72,55,178
5,1,Piotrków,89,51,139
6,1,Radom,102,50,288
7,2,Kielce,98,42,-
8,2,Czestochowa,79,40,278
9,2,Bytom,76,31,163


In [3]:
area_df = pd.read_csv('data/area.csv')
area_df['Include at least'] = area_df['Include at least'].apply(lambda x: [group.split(';') for group in x.split('|')] if pd.notna(x) else [])
area_df['Include at most'] = area_df['Include at most'].apply(lambda x: [group.split(';') for group in x.split('|')] if pd.notna(x) else [])
area_df

,Area,Minimum cities in main line,Starting city,Ending city,Include at least,Include at most
0,1,4,Warszawa,Radom,[[Konin]],[]
1,2,5,Kielce,Gliwice,[],"[[Bytom, Sosnowiec, Katowice], [Wodzisław Śląs..."
2,3,5,Opole,Jelenia Góra,"[[Kalisz], [Poznań, Zielona Góra, Leszno]]",[]


### Parameters

In [4]:
A = cities_df['Area'].unique().tolist()
C = {a: cities_df[cities_df['Area'] == a]['City'].tolist() for a in A}

M = {a: area_df[area_df['Area'] == a]['Minimum cities in main line'].values[0] for a in A}
S = {a: area_df[area_df['Area'] == a]['Starting city'].values[0] for a in A}
E = {a: area_df[area_df['Area'] == a]['Ending city'].values[0] for a in A}

d = {(a, c1, c2): utils.euclidean((cities_df[cities_df['City'] == c1]['x'].values[0], cities_df[cities_df['City'] == c1]['y'].values[0]), (cities_df[cities_df['City'] == c2]['x'].values[0], cities_df[cities_df['City'] == c2]['y'].values[0])) for a in A for c1 in C[a] for c2 in C[a] if c1 != c2}

In [5]:
model = Model('Main line optimization')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-29


### Variables

In [6]:
x = model.addVars([(a, c1, c2) for a in A for c1 in C[a] for c2 in C[a] if c1 != c2], vtype=GRB.BINARY, name='x')
y = model.addVars([(a, c) for a in A for c in C[a]], vtype=GRB.BINARY, name='y')

### Objective function

In [7]:
model.setObjective(quicksum(d[(a, c1, c2)]*x[a, c1, c2] for a in A for c1 in C[a] for c2 in C[a] if c1 != c2), GRB.MINIMIZE)

### Starting and ending city constraint

In [8]:
for a in A:
    model.addConstr(y[a, S[a]] == 1, name=f'Starting city {a}')
    model.addConstr(y[a, E[a]] == 1, name=f'Ending city {a}')

### Satisfaction constraint

In [9]:
model.addConstrs((quicksum(y[a, c] for c in C[a]) >= M[a] for a in A), name='Minimum cities in main line')

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### Local growth constraint, Decentralization constraint, German connection constraint

In [10]:
for index, row in area_df.iterrows():
    area = row['Area']
    include_at_least = row['Include at least']

    if include_at_least:
        for group in include_at_least:
            model.addConstr(quicksum(y[area, c] for c in group) >= 1, name=f'Include at least {area}')

In [11]:
for index, row in area_df.iterrows():
    area = row['Area']
    include_at_most = row['Include at most']

    if include_at_most:
        for group in include_at_most:
            model.addConstr(quicksum(y[area, c] for c in group) <= 1, name=f'Include at most {area}')

### Connection constraint

In [19]:
m = 1
for a in A:
    for c1 in C[a]:
        model.addConstr(quicksum(x[a,c1,c2] + x[a,c2,c1] for c2 in C[a] if c2 != c1) <= m * y[a,c1])

In [20]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.1.0 23B92)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40 rows, 230 columns and 476 nonzeros
Model fingerprint: 0x326c2dd0
Variable types: 0 continuous, 230 integer (230 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]

Loaded MIP start from previous solve with objective 0


Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [23]:
if model.status == GRB.OPTIMAL:
    for a in A:
        for i in C[a]:
            if y[a, i].x > 0.5:
                print(f"Main line in area {a} passes through city {i}")
        for i in C[a]:
            for j in C[a]:
                try:
                    if x[a, i, j].x > 0.5:
                        print(f"Main line in area {a} connects cities {i} and {j}")
                except KeyError:
                    pass

Main line in area 1 passes through city Warszawa
Main line in area 1 passes through city Skierniewice
Main line in area 1 passes through city Lódz
Main line in area 1 passes through city Konin
Main line in area 1 passes through city Sieradz
Main line in area 1 passes through city Piotrków
Main line in area 1 passes through city Radom
Main line in area 2 passes through city Kielce
Main line in area 2 passes through city Czestochowa
Main line in area 2 passes through city Gliwice
Main line in area 2 passes through city Kraków
Main line in area 2 passes through city Bielsko
Main line in area 3 passes through city Poznań
Main line in area 3 passes through city Zielona Góra
Main line in area 3 passes through city Leszno
Main line in area 3 passes through city Kalisz
Main line in area 3 passes through city Legnica
Main line in area 3 passes through city Wrocław
Main line in area 3 passes through city Jelenia Góra
Main line in area 3 passes through city Wałbrzych
Main line in area 3 passes th